# Demo Notebook for Tone Grabber

## Dataset Generator Example

Define the effects and effect parameter mappings for the dataset generation

In [ ]:
from dataset.data_generator import DataGenerator
from pedalboard import Reverb, Delay, Chorus, Distortion, Gain
# Dictionary of effects to parameter mappings
effects_to_parameters = {
    "Reverb": {
        "room_size": (0, 1),
        "damping": (0, 1), 
        "wet_level": (0, 1),
        "dry_level": (0, 1),
        "width": (0, 1),
        "freeze_mode": (0, 1)
    },
    "Delay": {
        "delay_seconds": (0, 2),
        "feedback": (0, 1),
        "mix": (0, 1)
    },
    "Chorus": {
        "rate_hz": (0, 100),
        "depth": (0, 1),
        "centre_delay_ms": (1, 30),
        "feedback": (0, 1),
        "mix": (0, 1)
    },
    "Distortion": {
        "drive_db": (0, 100)
    },
    "Gain": {
        "gain_db": (-12, 12)
    }
}
# List of effects
effects = [Reverb, Delay, Distortion, Gain, Chorus]

# create instance of data generator corresponding to effects
generator = DataGenerator(effects_to_parameters, effects)

create a demo dataset with

In [ ]:
import os
# num samples is the number of samples created per audio effect so total number of samples created will be:
# num_samples * number of dry_tones
num_samples = 2
audio_directory = os.path.join(os.getcwd(),"demo_data")
dry_tones = os.listdir(audio_directory)
# max_chain_length is the maximum number of effects applied to a sample
max_chain_length = 1
demo_dataset = generator.create_data(num_samples,audio_directory,dry_tones,max_chain_length)

Each entry of the dataset has this output signature:

```
"dry_tone": 
{
    "spectrogram":log mel spectrogram of the dry tone,
    "loudness":loudness of the dry tone,
    "f0":fundamental frequency of the dry tone,
    "path":path to the original dry tone
}
```

```
"wet_tone": {
    "spectrogram":log mel spectrogram of the wet tone,
    "loudness":loudness of the wet tone
    "f0":fundamental frequency of the wet tone,
    "path":path to the original wet tone
}
```
```
"effect_names":names of the applied effect(s)
```
```
"effects":one-hot encoding representation of the effects
```
```
"parameters": one-hot like representation of the effect parameters
```

In [ ]:
display(demo_dataset[0])

You can also get the metadata for the dataset

In [ ]:
metadata = generator.get_metadata()
display(metadata)

## Feature Extractor Demo

The feature extractor is built into the data generator class so it runs automatically when you run ```generator.create_data()``` 

But here is some demo code in case you run into problems using it anyway

In [ ]:
from pedalboard.io import ReadableAudioFile
from dataset.feature_extractor_torch import FeatureExtractorTorch
import numpy as np
# define instance of feature extractor
feature_extractor = FeatureExtractorTorch()
sample_rate = 16000
# read in audio path
dry_tone_path = "demo_data/guitar_acoustic_017-102-050.wav"
with ReadableAudioFile(dry_tone_path) as f:
    # re sample the audio file to match the sample rate, pretrained model is sampled at 16000
    re_sampled = f.resampled_to(sample_rate)
    dry_tone = np.squeeze(re_sampled.read(int(sample_rate * f.duration)),axis=0)
    re_sampled.close()
    f.close()
# read in features
features = feature_extractor.get_features(dry_tone)
# features extracted are log mel spectrogram, loudness, and fundamental frequency (f0)
display(features['spectrogram'].shape)
display(features['loudness'].shape)
display(features['f0'].shape)

## Parameter Prediction Demo

In [6]:
from model.parameter_prediction import ParameterPrediction
param_mask = metadata['parameter_mask_idx']
num_parameters = metadata['total_parameters']
num_effects = len(metadata['effect_to_idx'].keys())
model = ParameterPrediction(num_effects,num_parameters,param_mask)

In [ ]:
param_mask

In [ ]:
entry = demo_dataset[0]
wet_tone_feat = entry["wet_tone"]
dry_tone_feat = entry["dry_tone"]

out,  effect, params = model(wet_tone_feat['spectrogram'],dry_tone_feat['spectrogram'],wet_tone_feat['loudness'],wet_tone_feat['f0'],dry_tone_feat['loudness'],dry_tone_feat['f0'])
display(out.shape)
display(effect)
display(params)

### Interpolate output into pedalboad effect matched with parameters

first get the pedalboard effect object from the metadata

In [9]:
import torch
predicted_effect_pb = metadata['effects'][int(torch.argmax(effect))]

then match pedicted parameters to their names so  we can use them as input to the effect object

In [ ]:
predicted_params = [float(param) for param in list(params[0].detach()) if param != 0]
param_names = metadata['effects_to_parameters'][metadata['index_to_effect'][int(torch.argmax(effect))]].keys()
matched_params = {param_name:value for param_name,value in zip(param_names,predicted_params)}
matched_params

we can use this on the dry tone create the pedicted wet tone, since this model has random weights it will likely not be accurate rright now but hopefully it will be when trained

In [11]:
from pedalboard import Pedalboard
pred = predicted_effect_pb(**matched_params)
pedalboard = Pedalboard([pred])

In [12]:
from pedalboard.io import ReadableAudioFile
import numpy as np
with ReadableAudioFile(entry['dry_tone']['path']) as f:
    re_sampled = f.resampled_to(sample_rate)
    dry_tone = np.squeeze(re_sampled.read(int(sample_rate * f.duration)),axis=0)
    re_sampled.close()
    f.close()
wet_tone = pedalboard(dry_tone, sample_rate * f.duration)

#### Predicted wet tone

In [ ]:
import IPython.display as ipd
ipd.Audio(data=wet_tone,rate=sample_rate)

## Train the parameter prediction model on a sample dataset

In [ ]:
from model.parameter_prediction import Trainer
from auraloss.freq import MultiResolutionSTFTLoss
from torch.optim import Adam
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch.utils.data import DataLoader

train_loader = DataLoader(demo_dataset, batch_size=2, shuffle=True)
loss_fn = MultiResolutionSTFTLoss()
optimizer = Adam(model.parameters(),.0001)
scheduler = ReduceLROnPlateau(optimizer, 'min', patience=2, factor=0.25)
trainer = Trainer(model, metadata)
trainer.train(model, train_loader, train_loader, loss_fn, optimizer, scheduler, 5)

In [ ]:
for batch in train_loader:
    print(batch['dry_tone'])
    print(batch['wet_tone'])
    break